In [1]:
import sys
sys.path.append('/Users/zaneselvans/code/catalyst/pudl')
sys.path.append('/Users/christinagosnell/code/pudl')
sys.path.append('/Users/Nannerz/Desktop/working/pudl/')
from pudl import ferc1, pudl, models, models_ferc1, settings, constants
import numpy as np
import pandas as pd
import sqlalchemy as sa
from sqlalchemy import create_engine
from sqlalchemy.sql import select
%load_ext autoreload
%autoreload 1

In [2]:
%aimport pudl.pudl
%aimport pudl.ferc1
%aimport pudl.models
%aimport pudl.models_ferc1
%aimport pudl.eia923
%aimport pudl.constants

In [3]:
ferc1_engine = pudl.ferc1.db_connect_ferc1()
pudl.ferc1.init_db(refyear=2015, years=range(2004,2016), def_db=True)

Defining new FERC Form 1 DB based on 2015...
Ingesting FERC Form 1 Data from 2004...
Ingesting FERC Form 1 Data from 2005...
Ingesting FERC Form 1 Data from 2006...
Ingesting FERC Form 1 Data from 2007...
Ingesting FERC Form 1 Data from 2008...
Ingesting FERC Form 1 Data from 2009...
Ingesting FERC Form 1 Data from 2010...
Ingesting FERC Form 1 Data from 2011...
Ingesting FERC Form 1 Data from 2012...
Ingesting FERC Form 1 Data from 2013...
Ingesting FERC Form 1 Data from 2014...
Ingesting FERC Form 1 Data from 2015...


In [33]:
ferc1_respondents = pd.read_sql('''SELECT respondent_id, respondent_name FROM f1_respondent_id''', ferc1_engine)
ferc1_steam_plants = pd.read_sql('''SELECT respondent_id, plant_name, report_year, tot_capacity
                                    FROM f1_steam''', ferc1_engine)
ferc1_small_plants = pd.read_sql('''SELECT respondent_id, plant_name, report_year, kind_of_fuel, capacity_rating
                                    FROM f1_gnrt_plant''', ferc1_engine)
ferc1_hydro_plants = pd.read_sql('''SELECT respondent_id, plant_name, report_year, plant_kind, tot_capacity
                                    FROM f1_hydro''', ferc1_engine)
ferc1_pumped_storage_plants = pd.read_sql('''SELECT respondent_id, plant_name, report_year, plant_kind, tot_capacity
                                    FROM f1_pumped_storage''', ferc1_engine)

In [45]:
ferc1_plants_all_years = pd.concat([ferc1_steam_plants, ferc1_small_plants, ferc1_hydro_plants, ferc1_pumped_storage_plants])
ferc1_plants_all_years = ferc1_respondents.merge(ferc1_plants_all_years, how='left', on='respondent_id')
ferc1_plants_all_years = ferc1_plants_all_years.replace('^$', np.nan, regex=True)
ferc1_plants_all_years = ferc1_plants_all_years.dropna(subset=['plant_name',])
ferc1_plants_all_years['report_year'] = ferc1_plants_all_years.report_year.astype(int)
ordered_for_cgosnell = ['respondent_id', 'respondent_name', 'plant_name','report_year',]\
                        #'kind_of_fuel',  'capacity_rating', 'plant_kind', 'tot_capacity']
ferc1_plants_all_years = ferc1_plants_all_years[ordered_for_cgosnell]
ferc1_plants_all_years = ferc1_plants_all_years.drop_duplicates(subset=['respondent_id','plant_name'])
ferc1_plants_all = ferc1_plants_all_years[['respondent_id', 'respondent_name', 'plant_name']]

In [46]:
ferc1_plants_2015 = ferc1_plants_all_years.loc[ferc1_plants_all_years['report_year'] == 2015]
ferc1_plants_2015 = ferc1_plants_2015[['respondent_id', 'respondent_name', 'plant_name']]

In [47]:
ferc1_plants_2015.count()

respondent_id      184
respondent_name    184
plant_name         184
dtype: int64

In [48]:
ferc1_plants_all.count()

respondent_id      3499
respondent_name    3499
plant_name         3499
dtype: int64

In [49]:
ferc1_plants_2015 = set([ tuple(line) for line in ferc1_plants_2015.values.tolist()])
ferc1_plants_all = set([ tuple(line) for line in ferc1_plants_all.values.tolist()])
ferc1_plants_other_years = pd.DataFrame(list(ferc1_plants_all.difference(ferc1_plants_2015)))

In [50]:
ferc1_plants_other_years.count()

0    3315
1    3315
2    3315
dtype: int64

In [ ]:
ferc1_plants_all.to_csv('../../results/id_mapping/ferc1_all_plants_2004-2015.csv', index=False)

In [ ]:
ferc1_respondents.to_csv('../../results/id_mapping/ferc1_all_respondents_2004-2015.csv', index=False)

In [ ]:
import os.path
map_eia923_ferc1_file = os.path.join(settings.PUDL_DIR,
                                         'results',
                                         'id_mapping',
                                         'mapping_eia923_ferc1.xlsx')

In [ ]:
plant_map = pd.read_excel(map_eia923_ferc1_file,'plants_output',
                          na_values='', keep_default_na=False,
                          converters={'plant_id':int,
                                      'plant_name':str,
                                      'respondent_id_ferc1':int,
                                      'respondent_name_ferc1':str,
                                      'plant_name_ferc1':str,
                                      'plant_id_eia923':int,
                                      'plant_name_eia923':str,
                                      'operator_name_eia923':str,
                                      'operator_id_eia923':int})

In [ ]:
plants_ferc1 = plant_map[['respondent_id_ferc1','respondent_name_ferc1','plant_name_ferc1']]
plants_ferc1 = plants_ferc1.drop_duplicates(['plant_name_ferc1','respondent_id_ferc1'])

In [ ]:
plants_ferc1.count()

In [ ]:
ferc1_all_plants.sample(1)

In [ ]:
plants_ferc1 = set([ tuple(line) for line in plants_ferc1.values.tolist()])
ferc1_all_plants= set([ tuple(line) for line in ferc1_all_plants.values.tolist()])

In [ ]:
plants_other_years = pd.DataFrame(list(plants_ferc1.difference(ferc1_all_plants)))

In [ ]:
plants_other_years.count()

In [ ]:
plants_other_years.sample(5)

In [ ]:
plants_other_years.to_csv('../../results/id_mapping/plants_ferc1_2004-2014.csv', index=False)